In [1]:
import syft as sy

In [2]:
import os
#from google.colab import drive
#drive.mount('/content/drive')
# import syft as sy
# from syft.frameworks.torch.fl import utils
# from syft.workers.websocket_client import WebsocketClientWorker

import torch
import torchaudio
import torch.nn as nn
from tqdm import tqdm
from difflib import SequenceMatcher
from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import TensorDataset, DataLoader
import time
from librosa.core import stft, magphase
from torch.autograd import Variable
from data_load import CodeSwitchDataset, TextTransform
import csv 
import copy
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import librosa
import pandas as pd 
import cv2
import matplotlib.pyplot as plt 
from sklearn.preprocessing import MinMaxScaler 
import numpy as np

/home/dexter/Desktop/Projects/CodeSwitching/hparam.py:11: YAMLLoadWarning: calling yaml.load_all() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  for doc in docs:


tsv_file = open("/content/Transcription_LT_Sequence_Frame_Level_200_actual.tsv")
read_tsv = csv.reader(tsv_file, delimiter="\t")

for row in read_tsv:
  print(row)

In [3]:
train_audio_transforms = nn.Sequential(
            torchaudio.transforms.MelSpectrogram(n_mels=128, sample_rate = 22050, n_fft = 512, win_length=int(22050*0.02), hop_length=int(22050*0.01)),
            torchaudio.transforms.FrequencyMasking(freq_mask_param=15),
            torchaudio.transforms.TimeMasking(time_mask_param=35)
        )


valid_audio_transforms = torchaudio.transforms.MelSpectrogram()

text_transform = TextTransform()


def data_processing(data, data_type="train"):
    spectrograms = []
    labels = []
    input_lengths = []
    label_lengths = []
    for (waveform, _, utterance, _) in data:
        waveform=torch.Tensor(waveform)
        if data_type == 'train':
            spec = train_audio_transforms(waveform).squeeze(0).transpose(0, 1)
        else:
            spec = valid_audio_transforms(waveform).squeeze(0).transpose(0, 1)
        spectrograms.append(spec)
        label = torch.Tensor(text_transform.text_to_int(utterance.lower()))
        labels.append(label)
        input_lengths.append(spec.shape[0]//2)
        label_lengths.append(len(label))

    spectrograms = nn.utils.rnn.pad_sequence(spectrograms, batch_first=True).unsqueeze(1).transpose(2, 3)
    labels = nn.utils.rnn.pad_sequence(labels, padding_value = 1, batch_first=True)

    return spectrograms, labels, input_lengths, label_lengths

In [ ]:
def makeNonFedData():
  dcm_path=os.listdir('/content/drive/My Drive/guj/') #Gujurati-English Dataset
  dcm_data={}
  alpha = 1.5 
  beta = 0
  labels=[]
  pid=[]
  matrix=[]
  label=[]
  for file in dcm_path:
    name = '/content/drive/My Drive/minibatch_large/' + file
    temp = pymatrix.dcmread(name)
    image = temp.pixel_array
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    adjusted = cv2.resize(image,(128,128))
    dcm_data[file]=adjusted  
  with open('/content/drive/My Drive/stage_2_train_labels.csv','r') as file:
    reader = csv.reader(file)
    for row in reader:
      labels.append(row)  
  scaler = MinMaxScaler()
  cid = 0
  ### Bug - stft
  
  return matrix, label

In [ ]:
matrix, label = makeNonFedData()

In [ ]:
print(len(matrix),len(label))

4270 4270


In [4]:
class Arguments():
    def __init__(self):
        self.batch_size = 4
        self.test_batch_size = 4
        self.epochs = 60
        self.lr = 5e-4
        self.log_interval = 10
args = Arguments()
hparams = {
        "n_cnn_layers": 4,
        "n_rnn_layers": 5,
        "rnn_dim": 512,
        "n_class": 4,
        "n_feats": 128,
        "stride": 2,
        "dropout": 0.1,
        "learning_rate": 5e-4,
        "batch_size": 4,
        "epochs": 60
    }

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(matrix,label,test_size=0.3)
test_df = pd.DataFrame()
test_df['features']=x_test
test_df['labels']=y_test
x_train= np.array(x_train)
y_train= np.array(y_train)
x_test= np.array(x_test)
y_test= np.array(y_test)

In [9]:
train_dataset = CodeSwitchDataset(lang='Telugu', mode="train")
validation_split = 0.2
shuffle_dataset = True
random_seed = 42
dataset_size = len(train_dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset:
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

In [ ]:
class data_maker(Dataset):
    def __init__(self, images, labels):
        self.data = images
        self.targets = labels
        self.to_torchtensor()
        
    def __len__(self):
        return len(self.data)

    def to_torchtensor(self):      
        self.data=torch.from_numpy(self.data)
        self.labels=torch.from_numpy(self.targets)

    def __getitem__(self, idx):
        sample=self.data[idx]
        target=self.targets[idx]
        return sample,target

In [6]:
hook = sy.TorchHook(torch)
client1 = sy.VirtualWorker(hook, id="client1")
client2 = sy.VirtualWorker(hook, id="client2")

In [ ]:
federated_train_data = data_maker(x_train,y_train).federate((client1,client2))

In [11]:
# federated_train_loader = sy.FederatedDataLoader(federated_train_data,batch_size=args.batch_size)
BATCH_SIZE = 4
federated_train_loader = sy.FederatedDataLoader(train_dataset.federate((client1, client2)),
                                               batch_size=BATCH_SIZE,
                                                drop_last=True,
                                                num_workers = 6,
                                                sampler = train_sampler,
                                                collate_fn = lambda x: data_processing(x, 'train'))

test_loader = DataLoader(train_dataset,
                          batch_size=BATCH_SIZE,
                          drop_last=True,
                          num_workers=6,
                          sampler=valid_sampler,
                         collate_fn=lambda x: data_processing(x, 'valid'))

#test_data = data_maker(x_test,y_test)
#test_loader = torch.utils.data.DataLoader(test_data, batch_size=args.test_batch_size)

KeyboardInterrupt: 

In [12]:
class CNNLayerNorm(nn.Module):
    """Layer normalization built for cnns input"""
    def __init__(self, n_feats):
        super(CNNLayerNorm, self).__init__()
        self.layer_norm = nn.LayerNorm(n_feats)

    def forward(self, x):
        # x (batch, channel, feature, time)
        x = x.transpose(2, 3).contiguous() # (batch, channel, time, feature)
        x = self.layer_norm(x)
        return x.transpose(2, 3).contiguous() # (batch, channel, feature, time) 


class ResidualCNN(nn.Module):
    """Residual CNN inspired by https://arxiv.org/pdf/1603.05027.pdf
        except with layer norm instead of batch norm
    """
    def __init__(self, in_channels, out_channels, kernel, stride, dropout, n_feats):
        super(ResidualCNN, self).__init__()

        self.cnn1 = nn.Conv2d(in_channels, out_channels, kernel, stride, padding=kernel//2)
        self.cnn2 = nn.Conv2d(out_channels, out_channels, kernel, stride, padding=kernel//2)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.layer_norm1 = CNNLayerNorm(n_feats)
        self.layer_norm2 = CNNLayerNorm(n_feats)

    def forward(self, x):
        residual = x  # (batch, channel, feature, time)
        x = self.layer_norm1(x)
        x = F.gelu(x)
        x = self.dropout1(x)
        x = self.cnn1(x)
        x = self.layer_norm2(x)
        x = F.gelu(x)
        x = self.dropout2(x)
        x = self.cnn2(x)
        x += residual
        return x # (batch, channel, feature, time)


class BidirectionalGRU(nn.Module):

    def __init__(self, rnn_dim, hidden_size, dropout, batch_first):
        super(BidirectionalGRU, self).__init__()

        self.BiGRU = nn.GRU(
            input_size=rnn_dim, hidden_size=hidden_size,
            num_layers=1, batch_first=batch_first, bidirectional=True)
        self.layer_norm = nn.LayerNorm(rnn_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.layer_norm(x)
        x = F.gelu(x)
        x, _ = self.BiGRU(x)
        x = self.dropout(x)
        return x


class SpeechRecognitionModel(nn.Module):
    """Speech Recognition Model Inspired by DeepSpeech 2"""

    def __init__(self, n_cnn_layers, n_rnn_layers, rnn_dim, n_class, n_feats, stride=2, dropout=0.1):
        super(SpeechRecognitionModel, self).__init__()
        n_feats = n_feats//2
        self.cnn = nn.Conv2d(1, 64, 3, stride=stride, padding=3//2)  # cnn for extracting heirachal features

        # n residual cnn layers with filter size of 32
        self.rescnn_layers = nn.Sequential(*[

            ResidualCNN(64, 64, kernel=3, stride=1, dropout=dropout, n_feats=n_feats) 
            for _ in range(n_cnn_layers)
        ])
        self.fully_connected = nn.Linear(n_feats*64, rnn_dim)
        self.birnn_layers = nn.Sequential(*[
            BidirectionalGRU(rnn_dim=rnn_dim if i==0 else rnn_dim*2,
                             hidden_size=rnn_dim, dropout=dropout, batch_first=i==0)
            for i in range(n_rnn_layers)
        ])
        self.classifier = nn.Sequential(
            nn.Linear(rnn_dim*2, rnn_dim),  # birnn returns rnn_dim*2
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(rnn_dim, n_class)
        )

    def forward(self, x):
        x = self.cnn(x)
        x = self.rescnn_layers(x)
        sizes = x.size()
        x = x.view(sizes[0], sizes[1] * sizes[2], sizes[3])  # (batch, feature, time)
        x = x.transpose(1, 2) # (batch, time, feature)
        x = self.fully_connected(x)
        x = self.birnn_layers(x)
        x = self.classifier(x)
        return x

In [ ]:
model = Net().double()

In [ ]:
def train(model, device, train_loader, criterion, optimizer, epoch, iter_meter, writer, scheduler):
    model.train()
    data_len = len(train_loader.dataset)
    total_loss=0
    LR = 0
    train_loss = 0

    avg_acc = 0
    acc = []
    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    for batch_idx, (_data) in pbar:
        #bi, wav, label = batch_idx, wav, label
        for g in optimizer.param_groups:
            LR=g['lr']
        wav, labels, input_lengths, label_lengths = _data
        
        
        model = model.send(wav.location)
        # input_lengths, label_lengths = torch.IntTensor(input_lengths), torch.IntTensor(label_lengths)
        wav = wav.to(device)
        # wav = wav.float()
        labels = labels.to(device)
        optimizer.zero_grad()
        # output = model(wav, input_lengths)   #(batch, time, n_class) [4, 911, 3]
        output = model(wav)

        output = F.log_softmax(output, dim=2)
        output = output.transpose(0,1)
        
        # print(labels, label_lengths)
        loss = criterion(output, labels, input_lengths, label_lengths)
        loss.backward()
        optimizer.step()
        scheduler.step()
        #print(loss)
        total_loss+=loss
        iter_meter.step()
        
        model.get()
        decoded_preds, decoded_targets = GreedyDecoder(output.transpose(0, 1), labels, label_lengths)
        decoded_preds, decoded_targets = list(map(str.strip, decoded_preds)), list(map(str.strip, decoded_targets))
        print(decoded_preds, decoded_targets)
        print("preds: ", "".join(decoded_preds))
        for j in range(len(decoded_preds)):
            s = SequenceMatcher(None, decoded_targets[j], decoded_preds[j])
            acc.append(s.ratio())

        avg_acc = sum(acc)/len(acc)
        writer.add_scalar("train_accuracy", avg_acc, epoch)
        writer.add_scalar('train_loss', loss.item(), iter_meter.get())
        writer.add_scalar('CTCLoss', loss, epoch*len(train_loader)+1)
        writer.add_scalar('TLoss', total_loss, epoch*len(train_loader)+1)
        writer.add_scalar("Learning Rate", LR, epoch)
        if batch_idx % 100 == 0 or batch_idx == data_len:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(wav), data_len,
                    100. * batch_idx / len(train_loader), loss.item()))
            print("Train Accuracy: {}, Train loss: {}".format(avg_acc, train_loss))
    
    # for g in optimizer.param_groups:
    #   g['lr'] = g['lr']/LEARNING_ANNEAL
            
    #print(decoded_preds[0])
    if (epoch+1)%2 == 0:
        model.eval().cpu()
        ckpt_model_filename = "ckpt_epoch_" + str(epoch+1) + "_batch_id_" + str(batch_idx+1) + ".pth"
        ckpt_model_path = os.path.join("checkpoints/", ckpt_model_filename)
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            }, ckpt_model_path)
        model.to(device).train()


In [ ]:
def test(args, model,test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    #criterion = F.nll_loss()
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
model.to(device)
model.train()

for epoch in range(1, epochs+1):
    train(model, device, train_loader, criterion, optimizer, epoch, iter_meter, writer, scheduler)
    test(model, device, test_loader, criterion, epoch, writer)

model.eval().cpu()
save_model_filename = "final_epoch_" + str(epoch + 1)  + ".model"
save_model_path = os.path.join("checkpoints", save_model_filename)
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, save_model_path)

print("\nDone, trained model saved at", save_model_path)
